# QR Code Scanner with API Integration and Local Data


### 1. Importing Libraries

In [ ]:
import cv2
import requests
from pyzbar.pyzbar import decode
import pyttsx3
import pandas as pd
import csv

### 2. Defining API Functions

In [ ]:
def send_to_api(code):
    url = f"https://802b-197-53-109-195.ngrok-free.app/api/Product/{code}"
    response = requests.get(url)
    return response

def decode_qr_code(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    decoded_objects = decode(gray)
    if decoded_objects:
        return decoded_objects[0].data.decode('utf-8')
    else:
        return None

def speak(text):
    engine = pyttsx3.init()
    rate = engine.getProperty('rate')
    engine.setProperty('rate', rate - 45)
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[1].id)
    engine.say(text)
    engine.runAndWait()

### 3. Capturing and Processing QR Codes from Webcam

In [ ]:
def capture_and_process_qr():
    voice_rate_adjusted = False
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            qr_data = decode_qr_code(frame)
            if qr_data:
                # Send QR code data to your API
                response = send_to_api(qr_data)
                if response.status_code == 200:
                    data = response.json()
                    # Extract product information from API response (assuming structure)
                    product_name = data.get('product_name')
                    description = data.get('description')
                    price = data.get('price')
                    currency = data.get('currency')
                    # Construct product information text for TTS
                    product_text = f"Product Name: {product_name}\nDescription: {description}\nPrice: {price} {currency}"
                    # Speak the product information
                    speak(product_text)
                else:
                    print(f"Error: {response.status_code}")

            cv2.imshow('QR Code Scanner', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

capture_and_process_qr()